#### 다음 실습 코드는 학습 목적으로만 사용 바랍니다. 문의 : audit@korea.ac.kr 임성열 Ph.D.

In [1]:
%pip install torch numpy

  Using cached torch-2.7.1-cp311-none-macosx_11_0_arm64.whl.metadata (29 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
Using cached torch-2.7.1-cp311-none-macosx_11_0_arm64.whl (68.6 MB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
Using cached networkx-3.5-py3-none-any.whl (2.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [torch]32m2/3 [torch]
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# 데이터 준비
text = "hello world machine learning is fun "
chars = sorted(list(set(text)))  # 등장 문자 목록
char_to_idx = {c: i for i, c in enumerate(chars)}
idx_to_char = {i: c for i, c in enumerate(chars)}

# 파라미터 설정
input_size = len(chars)
hidden_size = 50
num_layers = 1
seq_length = 10
learning_rate = 0.01

# 학습 데이터 만들기
input_data = []
target_data = []

for i in range(0, len(text) - seq_length):
    input_seq = text[i:i+seq_length]
    target_seq = text[i+1:i+seq_length+1]
    
    input_data.append([char_to_idx[c] for c in input_seq])
    target_data.append([char_to_idx[c] for c in target_seq])

input_data = torch.LongTensor(input_data)
target_data = torch.LongTensor(target_data)

# 모델 정의
class CharLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(CharLSTM, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, input_size)

    def forward(self, x, hidden):
        x = self.embedding(x)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out)
        return out, hidden

model = CharLSTM(input_size, hidden_size, num_layers)

# 손실 함수 및 최적화
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 학습
for epoch in range(200):
    hidden = None
    output, hidden = model(input_data, hidden)
    
    loss = criterion(output.view(-1, input_size), target_data.view(-1))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 20 == 0:
        print(f"Epoch [{epoch}/200] Loss: {loss.item():.4f}")

# 문장 생성
def generate_text(start_str, length):
    model.eval()
    chars_input = torch.LongTensor([[char_to_idx[c] for c in start_str]])
    hidden = None
    result = start_str
    
    for _ in range(length):
        output, hidden = model(chars_input, hidden)
        last_char_logits = output[:, -1, :]
        last_char_idx = torch.argmax(last_char_logits, dim=1).item()
        
        result += idx_to_char[last_char_idx]
        
        chars_input = torch.LongTensor([[last_char_idx]])
    
    return result

# 결과 보기
print("\nGenerated Text:")
print(generate_text("hello", 50))


Epoch [0/200] Loss: 2.8350
Epoch [20/200] Loss: 0.2622
Epoch [40/200] Loss: 0.0948
Epoch [60/200] Loss: 0.0855
Epoch [80/200] Loss: 0.0833
Epoch [100/200] Loss: 0.0823
Epoch [120/200] Loss: 0.0817
Epoch [140/200] Loss: 0.0812
Epoch [160/200] Loss: 0.0808
Epoch [180/200] Loss: 0.0805

Generated Text:
hello world machine learning is fun fun fun fun fun fun
